In [1]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

/home/hadoop/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [27]:
user_info_train = pd.read_csv('../../pcredit/train/user_info_train.txt',
                                  header = None)
user_info_test = pd.read_csv('../../pcredit/test/user_info_test.txt',
                                 header = None)
# 设置字段（列）名称
col_names = ['userid', 'sex', 'occupation', 'education', 'marriage', 'household']
user_info_train.columns = col_names
user_info_test.columns = col_names
# 合并train、test
user_info = pd.concat([user_info_train, user_info_test])
user_info.head()

,userid,sex,occupation,education,marriage,household
0,3150,1,2,4,1,4
1,6965,1,2,4,3,2
2,1265,1,3,4,3,1
3,6360,1,2,4,3,2
4,2583,2,2,2,1,1


In [30]:
print user_info_train.describe(),user_info_test.describe()

            userid           sex    occupation     education      marriage  \
count  55596.00000  55596.000000  55596.000000  55596.000000  55596.000000   
mean   27798.50000      1.143967      2.382096      3.232858      1.997356   
std    16049.32712      0.428117      0.755043      0.766713      1.000050   
min        1.00000      0.000000      0.000000      0.000000      0.000000   
25%    13899.75000      1.000000      2.000000      3.000000      1.000000   
50%    27798.50000      1.000000      2.000000      3.000000      2.000000   
75%    41697.25000      1.000000      3.000000      4.000000      3.000000   
max    55596.00000      2.000000      4.000000      4.000000      5.000000   

          household  
count  55596.000000  
mean       2.343028  
std        1.181833  
min        0.000000  
25%        1.000000  
50%        2.000000  
75%        4.000000  
max        4.000000                userid           sex    occupation     education      marriage  \
count  13899.000000 

In [28]:
user_info.describe()

,userid,sex,occupation,education,marriage,household
count,69495.000000,69495.000000,69495.000000,69495.000000,69495.000000,69495.000000
mean,34748.000000,1.146744,2.383236,3.235916,1.974718,2.337190
std,20061.622816,0.431104,0.760686,0.769235,1.000695,1.182559
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17374.500000,1.000000,2.000000,3.000000,1.000000,1.000000
50%,34748.000000,1.000000,2.000000,3.000000,2.000000,2.000000
75%,52121.500000,1.000000,3.000000,4.000000,3.000000,4.000000
max,69495.000000,2.000000,4.000000,4.000000,5.000000,4.000000


In [29]:
user_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69495 entries, 0 to 13898
Data columns (total 6 columns):
userid        69495 non-null int64
sex           69495 non-null int64
occupation    69495 non-null int64
education     69495 non-null int64
marriage      69495 non-null int64
household     69495 non-null int64
dtypes: int64(6)
memory usage: 3.7 MB


In [3]:
user_info.index = user_info['userid']
user_info.head()

,userid,sex,occupation,education,marriage,household
userid,,,,,,
3150,3150,1,2,4,1,4
6965,6965,1,2,4,3,2
1265,1265,1,3,4,3,1
6360,6360,1,2,4,3,2
2583,2583,2,2,2,1,1


In [23]:
#  哑变量处理
dataDummy = user_info.head(10)
dataDummy['sex'].astype('category')
dummy = pd.get_dummies(dataDummy['sex']) #  获得哑变量的数据
dummy.head(2)

,1,2
userid,,
3150,1,0
6965,1,0


In [24]:
dummy = dummy.add_prefix('{}#'.format('sex'))  # 数据列名修改
dummy.head(2)

,sex#1,sex#2
userid,,
3150,1,0
6965,1,0


In [4]:
user_info.drop('userid',
                axis = 1,
                inplace = True)
user_info.head()

,sex,occupation,education,marriage,household
userid,,,,,
3150,1,2,4,1,4
6965,1,2,4,3,2
1265,1,3,4,3,1
6360,1,2,4,3,2
2583,2,2,2,1,1


In [5]:
# bank_detail
bank_detail_train = pd.read_csv('../../pcredit/train/bank_detail_train.txt',
                                    header = None)
bank_detail_test = pd.read_csv('../../pcredit/test/bank_detail_test.txt',
                                    header = None)
col_names = ['userid', 'tm_encode', 'trade_type', 'trade_amount', 'salary_tag']
bank_detail_train.columns = col_names
bank_detail_test.columns = col_names
bank_detail = pd.concat([bank_detail_train, bank_detail_test])  #  拼接数据集
# 在该数据集中，一个用户对应多条记录，这里我们采用对每个用户每种交易类型取均值进行聚合
bank_detail_n = (bank_detail.loc[:, ['userid', 'trade_type', 'trade_amount', 'tm_encode']])\
        .groupby(['userid', 'trade_type']).mean()

In [6]:
bank_detail_n.head()

trade_amount   tm_encode
userid trade_type                          
1      0              13.734079  5900125921
       1              13.255397  5900935322
3      0              13.249264  4767556643
       1               9.834236  5899778320
4      0              12.128566  5042088336

In [7]:
bank_detail_n = bank_detail_n.unstack()
bank_detail_n.head()

trade_amount                tm_encode              
trade_type            0          1             0             1
userid                                                        
1             13.734079  13.255397  5.900126e+09  5.900935e+09
3             13.249264   9.834236  4.767557e+09  5.899778e+09
4             12.128566  10.920132  5.042088e+09  5.902715e+09
9             14.371789  12.867697  5.004267e+09  5.910709e+09
10            12.720866  12.296680  5.912361e+09  5.911718e+09

In [8]:
bank_detail_n.columns = ['income', 'outcome', 'income_tm', 'outcome_tm']  #  修改列名
bank_detail_n.head(5)

,income,outcome,income_tm,outcome_tm
userid,,,,
1,13.734079,13.255397,5.900126e+09,5.900935e+09
3,13.249264,9.834236,4.767557e+09,5.899778e+09
4,12.128566,10.920132,5.042088e+09,5.902715e+09
9,14.371789,12.867697,5.004267e+09,5.910709e+09
10,12.720866,12.296680,5.912361e+09,5.911718e+09


In [9]:
browse_history_train = pd.read_csv('../../pcredit/train/browse_history_train.txt',
                                       header = None)
browse_history_test = pd.read_csv('../../pcredit/test/browse_history_test.txt',
                                       header = None)
col_names = ['userid', 'tm_encode_2', 'browse_data', 'browse_tag']
browse_history_train.columns = col_names
browse_history_test.columns = col_names
browse_history = pd.concat([browse_history_train, browse_history_test])
browse_history.head()

,userid,tm_encode_2,browse_data,browse_tag
0,34801,5926003545,173,1
1,34801,5926003545,164,4
2,34801,5926003545,38,7
3,34801,5926003545,45,1
4,34801,5926003545,110,7


In [10]:
# 这里采用计算每个用户总浏览行为次数进行聚合
browse_history_count = browse_history.loc[:, ['userid', 'browse_data']].groupby(['userid']).sum()
browse_history_count.head(5)

,browse_data
userid,
2,151500
3,38586
4,38764
6,1650
7,112420


In [11]:
# 分别处理完以上数据集后，根据userid进行join，方式选择‘outer'，没有bill或者bank数据的user在对应字段上将为Na值
loan_data = user_info.join(bank_detail_n, how = 'outer')

In [12]:
loan_data.head()

,sex,occupation,education,marriage,household,income,outcome,income_tm,outcome_tm
userid,,,,,,,,,
1,1,2,3,1,3,13.734079,13.255397,5.900126e+09,5.900935e+09
2,1,2,3,2,1,NaN,NaN,NaN,NaN
3,1,4,4,1,4,13.249264,9.834236,4.767557e+09,5.899778e+09
4,1,4,4,3,2,12.128566,10.920132,5.042088e+09,5.902715e+09
5,1,2,2,3,1,NaN,NaN,NaN,NaN


In [13]:
loan_data.tail()

,sex,occupation,education,marriage,household,income,outcome,income_tm,outcome_tm
userid,,,,,,,,,
69491,1,2,2,4,1,NaN,NaN,NaN,NaN
69492,0,2,4,1,4,NaN,NaN,NaN,NaN
69493,1,2,4,1,4,NaN,NaN,NaN,NaN
69494,2,2,3,1,2,NaN,NaN,NaN,NaN
69495,1,2,2,3,1,NaN,NaN,NaN,NaN
